In [1]:
%%HTML
<style>
   div#notebook-container    { width: 95%; }
   div#menubar-container     { width: 65%; }
   div#maintoolbar-container { width: 99%; }
</style>

In [2]:
import numpy as np
import pandas as pd
import lightgbm as lgb
import time

In [3]:
import os
import sys
sys.path.insert(0, os.path.abspath('../'))
from module.prepare_data import *

INFO: Pandarallel will run on 16 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [4]:
# def _make_lag_roll(base_test, target, shift_day, roll_wind):
#     # target, shift_day, roll_wind = LAG_DAY[0],LAG_DAY[1],LAG_DAY[2]
#     lag_df = base_test[['id','d',target]]
#     col_name = 'rolling_mean_tmp_'+str(shift_day)+'_'+str(roll_wind)
#     lag_df[col_name] = lag_df.groupby(['id'])[target].rolling(roll_wind).parallel_apply(np.mean).reset_index(0, drop=True)
#     print(lag_df[col_name].shape, lag_df.groupby(['id'])[col_name].transform(lambda x: x.shift(shift_day)).shape)
    
#     lag_df[col_name] = lag_df.groupby(['id'])[col_name].transform(lambda x: x.shift(shift_day))

#     # grid_df[f'rolling{i}_shift(shift_day)_fft_diff_amp_top{top_no}'] = grid_df.groupby(["id"])["sales_diff"].rolling(
#     #     i).parallel_apply(fft_peak).reset_index(0, drop=True)
#     # grid_df[f'rolling{i}_shift(shift_day)_fft_diff_amp_top{top_no}'] = grid_df.groupby(["id"])[
#     #     f'rolling{i}_shift(shift_day)_fft_diff_amp_top{top_no}'].transform(lambda x: x.shift(shift_day))
#     return lag_df[[col_name]]

In [14]:
END_TRAIN =  1913

In [6]:
if not os.path.exists(TEMP_FEATURE_PKL):
    train_df = pd.read_csv(f'{ORI_CSV_PATH}/sales_train_evaluation.csv')
#     train_df = train_df[train_df['id'].isin(IDS[:100])]
    prices_df = pd.read_csv(f'{ORI_CSV_PATH}/sell_prices.csv')
    calendar_df = pd.read_csv(f'{ORI_CSV_PATH}/calendar.csv')
    try:
        if not os.path.exists(BASE_PATH):
            os.makedirs(BASE_PATH)
    except OSError:
        print("Creation of the directory %s failed" % BASE_PATH)
    else:
        print("Successfully created the directory %s" % BASE_PATH)

    grid_df = extract_features(train_df, prices_df, calendar_df, target=TARGET, nan_mask_d=1913+28)
    
    grid_df['item_id'] = grid_df['item_id'].astype('category')
    grid_df['dept_id'] = grid_df['dept_id'].astype('category')
    grid_df['cat_id'] = grid_df['cat_id'].astype('category')
#     grid_df['item_id'] = grid_df['item_id'].apply(lambda x: int(x.split('_')[-1])).astype('category')
#     grid_df['dept_id'] = grid_df['dept_id'].apply(lambda x: int(x.split('_')[-1])).astype('category')
#     grid_df['cat_id'] = grid_df['cat_id'].replace({'HOBBIES': 0, 'HOUSEHOLD': 1, 'FOODS': 2}).astype('category')
#     for col in ['event_name_1', 'event_type_1', 'event_name_2', 'event_type_2']:
#         grid_df[col] = grid_df[col].replace(
#             dict(zip(grid_df[col].unique(), np.arange(grid_df[col].unique().shape[0])))).astype('category')
    grid_df = reduce_mem_usage(grid_df)
    grid_df.to_pickle(TEMP_FEATURE_PKL)
    del grid_df

Successfully created the directory ../cache/ver620
melt_train_df 47.10912537574768
extract_price_features 37.248167991638184
extract_calendar_features 71.56659507751465
extract_rolling_features 757.5818192958832
extract_encode_features 20.057915210723877
extract_sliding_shift_features 536.9259903430939
Mem. usage decreased to 7557.09 Mb (33.6% reduction)


In [7]:
grid_df = pd.read_pickle(TEMP_FEATURE_PKL)
grid_df['groups'] = grid_df['tm_y'].astype(str) + '_' + (grid_df['tm_m']//3).astype(str)
grid_df.to_pickle(TEMP_FEATURE_PKL)
del grid_df

In [8]:
useful_cols = dict(zip(STORES_IDS, [M5_FEATURES]*len(STORES_IDS)))

In [9]:
from module.lgbm_baseline import *

In [10]:
history_df = train_evaluate_model(useful_cols, TARGET, BASE_PATH)#stores_ids=['CA_1']
print(history_df.rmse_trn.mean(), history_df.rmse_val.mean(), history_df.rmse_diff.mean())

Train CA_1
Fold: 0


C:\Users\wangz\.conda\envs\tf_gpu\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[100]	training's rmse: 2.55244	valid_1's rmse: 2.7168
[200]	training's rmse: 2.43996	valid_1's rmse: 2.63446
[300]	training's rmse: 2.40508	valid_1's rmse: 2.61255
[400]	training's rmse: 2.38333	valid_1's rmse: 2.60171
[500]	training's rmse: 2.36491	valid_1's rmse: 2.59293
[600]	training's rmse: 2.34944	valid_1's rmse: 2.5875
[700]	training's rmse: 2.33517	valid_1's rmse: 2.58384
[800]	training's rmse: 2.32239	valid_1's rmse: 2.57984
[900]	training's rmse: 2.31123	valid_1's rmse: 2.57748
[1000]	training's rmse: 2.30199	valid_1's rmse: 2.57539
[1100]	training's rmse: 2.29205	valid_1's rmse: 2.57405
[1200]	training's rmse: 2.2828	valid_1's rmse: 2.57351
[1300]	training's rmse: 2.274	valid_1's rmse: 2.57247
[1400]	training's rmse: 2.26552	valid_1's rmse: 2.57173
Fold: 1
[100]	training's rmse: 2.56737	valid_1's rmse: 2.64068
[200]	training's rmse: 2.46575	valid_1's rmse: 2.5373
[300]	training's rmse: 2.431	valid_1's rmse: 2.51164
[400]	training's rmse: 2.40823	valid_1's rmse: 2.50019
[500]

C:\Users\wangz\.conda\envs\tf_gpu\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[100]	training's rmse: 2.0206	valid_1's rmse: 2.15767
[200]	training's rmse: 1.97497	valid_1's rmse: 2.12005
[300]	training's rmse: 1.95874	valid_1's rmse: 2.11326
[400]	training's rmse: 1.94819	valid_1's rmse: 2.11007
[500]	training's rmse: 1.93914	valid_1's rmse: 2.10871
[600]	training's rmse: 1.93169	valid_1's rmse: 2.10768
[700]	training's rmse: 1.92469	valid_1's rmse: 2.1073
[800]	training's rmse: 1.91804	valid_1's rmse: 2.10703
[900]	training's rmse: 1.91182	valid_1's rmse: 2.10678
[1000]	training's rmse: 1.90629	valid_1's rmse: 2.10717
[1100]	training's rmse: 1.90039	valid_1's rmse: 2.10728
[1200]	training's rmse: 1.89478	valid_1's rmse: 2.10752
[1300]	training's rmse: 1.88948	valid_1's rmse: 2.10788
[1400]	training's rmse: 1.88449	valid_1's rmse: 2.10817
Fold: 1
[100]	training's rmse: 2.04936	valid_1's rmse: 2.0816
[200]	training's rmse: 2.00784	valid_1's rmse: 2.04312
[300]	training's rmse: 1.99104	valid_1's rmse: 2.03507
[400]	training's rmse: 1.9791	valid_1's rmse: 2.03146
[

...

[100]	training's rmse: 1.73332	valid_1's rmse: 1.69496
[200]	training's rmse: 1.68312	valid_1's rmse: 1.67282
[300]	training's rmse: 1.66575	valid_1's rmse: 1.66906
[400]	training's rmse: 1.65433	valid_1's rmse: 1.66752
[500]	training's rmse: 1.64544	valid_1's rmse: 1.66644
[600]	training's rmse: 1.63731	valid_1's rmse: 1.66574
[700]	training's rmse: 1.63022	valid_1's rmse: 1.66502
[800]	training's rmse: 1.62335	valid_1's rmse: 1.66455
[900]	training's rmse: 1.6172	valid_1's rmse: 1.66413
[1000]	training's rmse: 1.61144	valid_1's rmse: 1.66389
[1100]	training's rmse: 1.60589	valid_1's rmse: 1.66373
[1200]	training's rmse: 1.60047	valid_1's rmse: 1.66359
[1300]	training's rmse: 1.59531	valid_1's rmse: 1.66377
[1400]	training's rmse: 1.59056	valid_1's rmse: 1.66394
Fold: 1
[100]	training's rmse: 1.70858	valid_1's rmse: 1.78991
[200]	training's rmse: 1.66255	valid_1's rmse: 1.74586
[300]	training's rmse: 1.64785	valid_1's rmse: 1.73517
[400]	training's rmse: 1.63852	valid_1's rmse: 1.7303

C:\Users\wangz\.conda\envs\tf_gpu\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[100]	training's rmse: 2.64682	valid_1's rmse: 2.80421
[200]	training's rmse: 2.58453	valid_1's rmse: 2.73881
[300]	training's rmse: 2.55979	valid_1's rmse: 2.72537
[400]	training's rmse: 2.54095	valid_1's rmse: 2.71804
[500]	training's rmse: 2.52565	valid_1's rmse: 2.71479
[600]	training's rmse: 2.51033	valid_1's rmse: 2.71211
[700]	training's rmse: 2.49766	valid_1's rmse: 2.71038
[800]	training's rmse: 2.48542	valid_1's rmse: 2.70902
[900]	training's rmse: 2.47401	valid_1's rmse: 2.70844
[1000]	training's rmse: 2.46354	valid_1's rmse: 2.70783
[1100]	training's rmse: 2.45262	valid_1's rmse: 2.70734
[1200]	training's rmse: 2.44255	valid_1's rmse: 2.7071
[1300]	training's rmse: 2.43296	valid_1's rmse: 2.70712
[1400]	training's rmse: 2.42323	valid_1's rmse: 2.7081
Fold: 1
[100]	training's rmse: 2.74552	valid_1's rmse: 2.57582
[200]	training's rmse: 2.66954	valid_1's rmse: 2.54023
[300]	training's rmse: 2.63918	valid_1's rmse: 2.53402
[400]	training's rmse: 2.61995	valid_1's rmse: 2.52997

C:\Users\wangz\.conda\envs\tf_gpu\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[100]	training's rmse: 2.39181	valid_1's rmse: 2.73527
[200]	training's rmse: 2.26869	valid_1's rmse: 2.6046
[300]	training's rmse: 2.22758	valid_1's rmse: 2.57201
[400]	training's rmse: 2.20436	valid_1's rmse: 2.55656
[500]	training's rmse: 2.1859	valid_1's rmse: 2.54889
[600]	training's rmse: 2.17065	valid_1's rmse: 2.54327
[700]	training's rmse: 2.15475	valid_1's rmse: 2.53877
[800]	training's rmse: 2.14143	valid_1's rmse: 2.53456
[900]	training's rmse: 2.12967	valid_1's rmse: 2.53381
[1000]	training's rmse: 2.1188	valid_1's rmse: 2.5325
[1100]	training's rmse: 2.10876	valid_1's rmse: 2.53308
[1200]	training's rmse: 2.09892	valid_1's rmse: 2.53306
[1300]	training's rmse: 2.08951	valid_1's rmse: 2.5332
[1400]	training's rmse: 2.07995	valid_1's rmse: 2.53317
Fold: 1
[100]	training's rmse: 2.46279	valid_1's rmse: 2.45106
[200]	training's rmse: 2.34534	valid_1's rmse: 2.34759
[300]	training's rmse: 2.29859	valid_1's rmse: 2.32235
[400]	training's rmse: 2.26863	valid_1's rmse: 2.31127
[5

In [95]:
history_df.to_pickle(f'{BASE_PATH}/lgbm_history_df.pkl')

In [64]:
final_all_preds = predict_test(useful_cols, TARGET, BASE_PATH, end_train=END_TRAIN)

FOLD0 Predict | Day:1
##########  3.32 min round |  3.32 min total |  36736.00 day sales |
FOLD0 Predict | Day:2
##########  3.32 min round |  6.64 min total |  34976.00 day sales |
FOLD0 Predict | Day:3
##########  3.32 min round |  9.96 min total |  34528.00 day sales |
FOLD0 Predict | Day:4
##########  3.31 min round |  13.27 min total |  35040.00 day sales |
FOLD0 Predict | Day:5
##########  3.34 min round |  16.61 min total |  41312.00 day sales |
FOLD0 Predict | Day:6
##########  3.34 min round |  19.95 min total |  50624.00 day sales |
FOLD0 Predict | Day:7
##########  3.35 min round |  23.30 min total |  55040.00 day sales |
FOLD0 Predict | Day:8
##########  5.36 min round |  28.66 min total |  44160.00 day sales |
FOLD0 Predict | Day:9
##########  5.39 min round |  34.06 min total |  44384.00 day sales |
FOLD0 Predict | Day:10
##########  5.41 min round |  39.46 min total |  39296.00 day sales |
FOLD0 Predict | Day:11
##########  5.38 min round |  44.84 min total |  40800.00 d

In [94]:
final_all_preds.to_pickle(f'{BASE_PATH}/lgbm_final_all_preds.pkl')

In [92]:
final_all_preds['id'] = final_all_preds['id'].apply(lambda x: x.replace('evaluation', 'validation'))
smaple = pd.read_csv(f'{ORI_CSV_PATH}/sample_submission.csv')
submission = pd.concat([final_all_preds ,smaple[30490:]], axis=0)
submission.to_csv(f'{BASE_PATH}/lgbm_baseline.csv', index=False)

In [96]:
final_all_preds = pd.read_pickle(f'{BASE_PATH}/lgbm_final_all_preds.pkl')

In [97]:
final_all_preds

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0.823730,0.749023,0.741699,0.774902,0.977539,1.131836,1.335938,0.891602,0.940918,...,0.859863,1.157227,1.088867,0.875488,0.784180,0.760742,0.819336,0.958984,1.183594,1.076172
1,HOBBIES_1_002_CA_1_validation,0.205200,0.191895,0.173828,0.189087,0.223511,0.290527,0.304443,0.237549,0.232422,...,0.257812,0.315430,0.320068,0.224976,0.208008,0.203491,0.221436,0.240967,0.306885,0.311523
2,HOBBIES_1_003_CA_1_validation,0.436035,0.400635,0.419678,0.421875,0.586914,0.741699,0.682617,0.496094,0.472656,...,0.623047,0.761230,0.776367,0.509277,0.470215,0.471436,0.488525,0.627930,0.727051,0.706055
3,HOBBIES_1_004_CA_1_validation,1.653320,1.343750,1.386719,1.542969,2.009766,2.734375,3.089844,1.714844,1.476562,...,1.892578,2.527344,3.083984,1.747070,1.501953,1.408203,1.413086,1.935547,2.921875,3.292969
4,HOBBIES_1_005_CA_1_validation,0.978516,0.872070,0.930176,0.965820,1.132812,1.375000,1.554688,1.144531,1.127930,...,1.103516,1.537109,1.625977,1.021484,0.917969,0.926758,0.930664,1.136719,1.529297,1.538086
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_validation,0.379395,0.343750,0.344238,0.346680,0.390137,0.449463,0.517578,0.540039,0.594238,...,0.487061,0.655273,0.760254,0.510742,0.442139,0.414062,0.375977,0.432617,0.499756,0.526367
30486,FOODS_3_824_WI_3_validation,0.300049,0.281250,0.274170,0.262451,0.308838,0.328125,0.348633,0.372314,0.372070,...,0.285645,0.405762,0.436523,0.300293,0.238159,0.224121,0.212891,0.216309,0.281738,0.283447
30487,FOODS_3_825_WI_3_validation,0.601562,0.503906,0.497070,0.506836,0.625000,0.777344,0.981445,1.088867,1.078125,...,0.939941,1.290039,1.474609,1.000000,0.706543,0.708008,0.596680,0.668945,0.819824,0.824707
30488,FOODS_3_826_WI_3_validation,0.938965,0.889648,0.800781,0.812500,0.946289,1.255859,1.191406,1.184570,1.240234,...,1.076172,1.431641,1.473633,1.061523,0.968750,0.888184,0.886230,1.029297,1.126953,1.242188


In [99]:
submission

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0.823730,0.749023,0.741699,0.774902,0.977539,1.131836,1.335938,0.891602,0.940918,...,0.859863,1.157227,1.088867,0.875488,0.784180,0.760742,0.819336,0.958984,1.183594,1.076172
1,HOBBIES_1_002_CA_1_validation,0.205200,0.191895,0.173828,0.189087,0.223511,0.290527,0.304443,0.237549,0.232422,...,0.257812,0.315430,0.320068,0.224976,0.208008,0.203491,0.221436,0.240967,0.306885,0.311523
2,HOBBIES_1_003_CA_1_validation,0.436035,0.400635,0.419678,0.421875,0.586914,0.741699,0.682617,0.496094,0.472656,...,0.623047,0.761230,0.776367,0.509277,0.470215,0.471436,0.488525,0.627930,0.727051,0.706055
3,HOBBIES_1_004_CA_1_validation,1.653320,1.343750,1.386719,1.542969,2.009766,2.734375,3.089844,1.714844,1.476562,...,1.892578,2.527344,3.083984,1.747070,1.501953,1.408203,1.413086,1.935547,2.921875,3.292969
4,HOBBIES_1_005_CA_1_validation,0.978516,0.872070,0.930176,0.965820,1.132812,1.375000,1.554688,1.144531,1.127930,...,1.103516,1.537109,1.625977,1.021484,0.917969,0.926758,0.930664,1.136719,1.529297,1.538086
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60975,FOODS_3_823_WI_3_evaluation,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
60976,FOODS_3_824_WI_3_evaluation,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
60977,FOODS_3_825_WI_3_evaluation,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
60978,FOODS_3_826_WI_3_evaluation,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [ ]:
submission = pd.read_csv(f'{BASE_PATH}/lgbm_baseline.csv')